In [1]:
import stereo as st
import pandas as pd
import bokeh
import matplotlib.pyplot as plt

/opt/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# LISTA data (DY1_D0_stereo-seq.h5ad)

## Area

DY1_D0 represents $48.143 mm^2$

From the paper: each bin contains 50x50 nanoballs, so the h5ad bin size is incorrect.

Center-to-center distance of 715nm between nanoballs, aka bin-resolution of 36x36 um

Liver cells are larger, so this is probably capturing close to a single-cell diameter.

## Design

In total, 30 whole liver lobe sections

Time series (Day 0,1,2,3,7)

Matched scRNA-seq.

# Find anndata under 'adata'

In [ ]:
data = st.io.read_h5ad('../data/DY1_D0_stereo-seq.h5ad')

In [ ]:
# anndata object is under adata
data.adata

In [ ]:
data.adata.X.shape

In [ ]:
data.adata.var

In [ ]:
data.adata.obs

In [ ]:
# These two ways of accessing the counts matrix are equivalent
data.exp_matrix == h5ad.adata.X

In [ ]:
# Here are the bin positions
data.position

In [ ]:
# Bin size variable in here is not correct
data.bin_size

In [ ]:
# Setting up a 1mm scalebar we can add to our plots

def add_scalebar(ax, spot_to_spot_distance = 0.5):

    from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar

    # spot_to_spot_distance = 0.5 # um
    scale_bar_size = 500 # um (1mm)
    length = scale_bar_size / spot_to_spot_distance

    scalebar = AnchoredSizeBar(ax.transData,
        scale_bar_size / spot_to_spot_distance, '500um', 'lower right', 
        pad=0.4,
        color='black',
        frameon=False,
        size_vertical=length / 6)
    
    ax.add_artist(scalebar)

In [ ]:
data.adata.shape

In [ ]:
data.adata.shape[0] * data.adata.shape[1] * 4

In [ ]:
# Get approx size in GB of this data to check that it makes sense
4380904700 / 1_000_000_000

## Visualize the summed expression

In [ ]:
viz_data = pd.DataFrame(data.position, columns=['X','Y'])
viz_data['counts'] = data.exp_matrix.sum(axis=1)

In [ ]:
ax = viz_data \
    [lambda x: x.counts > 0] \
    .plot.scatter('X', 'Y', c='counts', s=2, cmap='viridis', marker='s')

# Despite what h5ad.bin_size indicates,
# This data is definitely binned.
# Going from the actual paper, it appears to be bin70 (go figure)
# i.e., 70 * 

add_scalebar(ax, spot_to_spot_distance=0.5*70)
ax.set_aspect('equal') # Keep aspect ratio fixed

plt.gcf().set_size_inches(10,8)

In [ ]:
data.adata.uns['annotation_colors']

In [ ]:
# This is the precalculated clusters
data.adata.obs['annotation']

In [ ]:
data

In [ ]:
## Read in data

In [ ]:
import stereo as st

In [2]:
data = st.io.read_h5ad("../data/DY1_D0_stereo-seq.h5ad")

In [3]:
data

AnnData object with n_obs × n_vars = 37669 × 29075
    obs: 'annotation'
    uns: 'annotation_colors', 'sn'
    obsm: 'spatial'

## Data QC

This function will calculate QC based on 3 metrics:

total_counts - the total counts per cell;

n_genes_by_counts - the number of genes expressed in count maxtrix;

pct_countss_mt - the percentage of counts in mitochondrial genes.

In [4]:
# Preprocessing
data.tl.cal_qc()


[2024-11-06 21:38:59][Stereo][3160][MainThread][140009698979840][st_pipeline][41][INFO]: start to run cal_qc...
[2024-11-06 21:39:04][Stereo][3160][MainThread][140009698979840][st_pipeline][44][INFO]: cal_qc end, consume time 5.4145s.


In [ ]:
# Visualise the QC results

In [ ]:
data.plt.violin()

In [ ]:
# Visualise data overlayed on sample

data.plt.spatial_scatter()


## Filtering

In [ ]:
data.plt.genes_count()


In [ ]:
data

In [ ]:
data.tl.filter_cells(
        min_counts=200,
        min_genes=3,
        pct_counts_mt=4,
        inplace=True
        )
data

## Normalisation

In [ ]:
#data.tl.raw_checkpoint()
#data.tl.raw
#data.tl.sctransform(res_key='sctransform', n_genes=2000, filter_hvgs=False, inplace=True)
data.tl.normalize_total(target_sum=10000)
data.tl.log1p()

## Highly variable genes

In [ ]:
#Highly variable genes
data.tl.highly_variable_genes(min_mean=0.0125, max_mean=3,min_disp=0.5, n_top_genes=2000, res_key='highly_variable_genes')
data.tl.scale()


In [ ]:
#  analysis of spatial hotspot
data.tl.spatial_hotspot(
                    use_highly_genes=True,
                    use_raw=True,
                    hvg_res_key='highly_variable_genes',
                    model='normal',
                    n_neighbors=30,
                    n_jobs=20,
                    fdr_threshold=0.05,
                    min_gene_threshold=10,
                    res_key='spatial_hotspot',
                    )

In [ ]:
data.tl.pca(
        use_highly_genes=True,
        n_pcs=30,
        res_key='pca'
        )

## Clustering

In [ ]:
data.tl.neighbors(pca_res_key='pca', res_key='neighbors')


In [ ]:
data.tl.umap(
        pca_res_key='pca',
        neighbors_res_key='neighbors',
        res_key='umap'
        )

In [ ]:
data.plt.umap(gene_names=['Atpif1', 'Tmsb4x'], res_key='umap')


In [ ]:
data.tl.leiden(neighbors_res_key='neighbors',res_key='leiden')


In [ ]:
data.plt.cluster_scatter(res_key='leiden')


In [ ]:
data.plt.cluster_scatter(res_key='leiden', groups=['1', '2'])


In [ ]:
#data.plt.cells_plotting(color_by='cluster', color_key='leiden')


In [ ]:
data.plt.interact_cluster(res_key='leiden')


In [ ]:
data.tl.find_marker_genes(
        cluster_res_key='leiden',
        method='t_test',
        use_highly_genes=False,
        use_raw=True
        )

In [ ]:
data.plt.marker_genes_text(
        res_key='marker_genes',
        markers_num=10,
        sort_key='scores'
        )

In [ ]:
data.plt.marker_genes_scatter(res_key='marker_genes', markers_num=5)


## Annotation

In [ ]:
data.plt.interact_annotation_cluster(
            res_cluster_key='leiden',
            res_marker_gene_key='marker_genes',
            res_key='leiden_annotation'
            )

In [ ]:
annotation_dict = {
    '1':'a',
    '2':'b',
    '3':'c',
    '4':'d',
    '5':'e',
    '6':'f',
    '7':'g',
    '8':'h',
    '9':'i',
    '10':'j',
    '11':'k',
    '12':'l',
    '13':'m',
    '14':'n',
    '15':'o',
    '16':'p',
    '17':'q',
    '18':'r',
    '19':'s',
    '20':'t',
    '21':'u',
    '22':'v',
    '23':'w',
    '24':'x',
    '25':'y'

}
data.tl.annotation(
        annotation_information=annotation_dict,
        cluster_res_key='leiden',
        res_key='anno_leiden'
        )

## Visualisation

In [ ]:
ins = data.plt.interact_spatial_scatter(width=500, height=500, poly_select=True)
ins.show()

## Annotation with single-cell reference

In [ ]:
ref_file = '../Homeostasis_scRNAseq.h5ad'

ref = st.io.read_h5ad(ref_file)

In [5]:
ref.tl.log1p()
ref.tl.normalize_total()

data.tl.cal_qc()
data.tl.log1p()
data.tl.normalize_total()

NameError: name 'ref' is not defined

In [ ]:
# gpu
data.tl.single_r(
    ref_exp_data=ref,
    ref_use_col='ClusterName',
    res_key='annotation',
    method='rapids'  #  Specifying the method as rapids means using gpu
)